In [1]:
%load_ext autoreload
%autoreload 1
%aimport read_atlas_layer_points 
#import read_atlas_layer_points as rlp

main_folder = 'C:/D/dev/data/20171205_cirrus_manual_results'
subfolders = ['CC_0003', 'CC_0005', 'CC_0006_', 'CC_0008']

filenames = read_atlas_layer_points.get_filenames(main_folder, subfolders)
print('Filenames: ', filenames)

Filenames:  ['C:/D/dev/data/20171205_cirrus_manual_results\\CC_0003\\deformed_layer_points.csv', 'C:/D/dev/data/20171205_cirrus_manual_results\\CC_0005\\deformed_layer_points.csv', 'C:/D/dev/data/20171205_cirrus_manual_results\\CC_0006_\\deformed_layer_points.csv', 'C:/D/dev/data/20171205_cirrus_manual_results\\CC_0008\\deformed_layer_points.csv']


In [78]:
# test read each file
layer9_points = read_atlas_layer_points.read_layer_points(filenames[0])
layer9_points['9']

array([[  1.03945005e+00,  -1.18992000e-03,  -3.55093986e-01],
       [  1.03937995e+00,   1.05100004e-02,  -3.55089992e-01],
       [  1.03930998e+00,   2.22098995e-02,  -3.55087012e-01],
       ..., 
       [  1.02127004e+00,   5.95231009e+00,   5.64107990e+00],
       [  1.02135003e+00,   5.96401978e+00,   5.64107990e+00],
       [  1.02137995e+00,   5.97571993e+00,   5.64107990e+00]], dtype=float32)

In [104]:
# read all file
atlas_layer_points = read_atlas_layer_points.get_atlas_layer_points(filenames, False)
print('Data shape: ', atlas_layer_points['1'].shape, ' number of points: ', 512*128*len(filenames))

Data shape:  (262144, 3)  number of points:  262144


In [8]:
import numpy as np

In [98]:
# compute normalization factor for points in each layer

def normalization_transform(points, verbose=False):
    """
        points is an numpy ndarray of size n x dim
        return 
            scale : scalar value
            centroid : 1 x 3 ndarray
    """
    n_pnts = points.shape[0]
    # find centroid
    if verbose: print(type(points))
    centroid = np.sum(points, axis = 0)/n_pnts
    # find scale
    distance2centroid = lambda pnt: pnt-centroid
    deviation = np.apply_along_axis(distance2centroid, axis=1, arr=points)
    scale = np.linalg.norm(deviation, ord='fro')/n_pnts
    if verbose: print(scale)
        
    return scale, centroid

In [99]:

label1_points = atlas_layer_points['1']
scale, centroid = normalization_transform(label1_points)
print('Scale ', scale, ' centroid ', centroid)

Scale  0.00480795243531  centroid  [ 0.73148078  2.99947762  2.88289165]


In [102]:
#apply normalization
def normalize_points(points, scale, centroid):
    relative2centroid = lambda pnt: pnt-centroid
    local_points = np.apply_along_axis(relative2centroid, axis=1, arr=points)
    normalized_points = local_points/scale
    return normalized_points

def denormalize_points(normalized_point, scale, centroid):
    points = normalized_point*scale
    relative2world = lambda pnt: pnt+centroid
    points = np.apply_along_axis(relative2world, axis=1, arr=points)
    return points

In [105]:
norm_l1_points = normalize_points(label1_points, scale, centroid)
denorm_l1_points = denormalize_points(norm_l1_points, scale, centroid)

print('Original \n', label1_points[:2, :], '\nDenorm \n', denorm_l1_points[:2, :])
print('Norm \n', norm_l1_points[:2, :])

Original 
 [[ 0.80515701 -0.00160219 -0.355315  ]
 [ 0.80447602  0.0100966  -0.35531199]] 
Denorm 
 [[ 0.80515701 -0.00160219 -0.355315  ]
 [ 0.80447602  0.0100966  -0.35531199]]
Norm 
 [[  15.32382641 -624.19082868 -673.51054287]
 [  15.1821896  -621.75761196 -673.50991681]]


In [106]:
from sklearn import preprocessing
layer1th_scaled = preprocessing.scale(atlas_layer_points['1'])

C:\Users\Ja\Anaconda3\envs\aind-dl\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\Ja\Anaconda3\envs\aind-dl\lib\site-packages\sklearn\preprocessing\data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [109]:
# test on label 1
number_of_train_files = 2
number_of_points_per_file = 512*128
X_trains = atlas_layer_points['1'][:2*number_of_points_per_file, 1:]
y_trains = atlas_layer_points['1'][:2*number_of_points_per_file, 0]

X_test = atlas_layer_points['1'][2*number_of_points_per_file:, 1:]
y_test = atlas_layer_points['1'][2*number_of_points_per_file:, 0]

print('X train: \n', X_trains[:10, :], '\nY train:\n', y_trains[:10])
print('X test: \n', X_test[:10, :], '\nY test:\n', y_test[:10])

X train: 
 [[-0.00160219 -0.355315  ]
 [ 0.0100966  -0.35531199]
 [ 0.0217955  -0.35530901]
 [ 0.0334946  -0.355306  ]
 [ 0.0451937  -0.35530299]
 [ 0.056893   -0.35530001]
 [ 0.0685922  -0.355297  ]
 [ 0.0802915  -0.35529301]
 [ 0.0919909  -0.35529   ]
 [ 0.10369    -0.35528699]] 
Y train:
 [ 0.80515701  0.80447602  0.80383098  0.80328101  0.80280399  0.80235201
  0.80192     0.80151099  0.80113101  0.80076998]
X test: 
 [[-0.0256177   0.070147  ]
 [-0.0139183   0.0701467 ]
 [-0.00221914  0.0701462 ]
 [ 0.00947993  0.0701457 ]
 [ 0.021179    0.0701451 ]
 [ 0.0328782   0.0701446 ]
 [ 0.0445774   0.0701442 ]
 [ 0.0562767   0.0701439 ]
 [ 0.067976    0.0701435 ]
 [ 0.0796753   0.0701432 ]] 
Y test:
 [ 0.81688201  0.81597102  0.80875498  0.80712301  0.80562103  0.80443197
  0.80348003  0.80267     0.801925    0.80127001]


In [114]:
from sklearn.externals import joblib
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import Matern

param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "kernel": [Matern(length_scale=l, nu=2.5) for l in np.logspace(-3, 0, 20)]}
kr = GridSearchCV(KernelRidge(), cv=5, param_grid=param_grid, n_jobs=-1, 
                  verbose=100, return_train_score=False)

In [115]:
import time
import os

t0 = time.time()
kr.fit(X_trains, y_trains)
print('KRR complexity and bandwidth selected and model fitted in %.3f s' % (time.time() - t0))

#save kernel
#filepath = os.path.join(main_folder, subfolder)
filename = os.path.join(main_folder, 'krr.pkl')
joblib.dump(kr, filename) 
# clf = joblib.load('filename.pkl') 

# View the accuracy score
print('Best score for data1:', kr.best_score_) 
# View the best parameters for the model found using grid search
print('Best Kernel:',kr.best_estimator_.kernel)
print('Best length scale:',kr.best_estimator_.length_scale)
print('Best length scale bound:',kr.best_estimator_.length_scale_bounds)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Pickling array (shape=(131072, 2), dtype=float32).
Pickling array (shape=(131072,), dtype=float32).
Pickling array (shape=(104857,), dtype=int32).
Pickling array (shape=(26215,), dtype=int32).
Pickling array (shape=(131072, 2), dtype=float32).
Pickling array (shape=(131072,), dtype=float32).
Pickling array (shape=(104857,), dtype=int32).
Pickling array (shape=(26215,), dtype=int32).
Pickling array (shape=(131072, 2), dtype=float32).
Pickling array (shape=(131072,), dtype=float32).
Pickling array (shape=(104858,), dtype=int32).
Pickling array (shape=(26214,), dtype=int32).


JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Ja\Anaconda3\envs\aind-dl\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...-dl\\lib\\site-packages\\ipykernel\\__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Ja\Anaconda3\env

In [ ]:
# Test accuracy : https://chrisalbon.com/machine-learning/cross_validation_parameter_tuning_grid_search.html
kr.score(X_test, y_test)

In [ ]:
# predict data: see http://scikit-learn.org/stable/auto_examples/gaussian_process/plot_compare_gpr_krr.html
# Given new X, determine corresponding y
#t0 = time.time()
#y_kr = kr.predict(X_plot)
#print("Time for KRR prediction: %.3f" % (time.time() - t0))